# T5rus с промптом + LoRA + псевдотекст

T5rus хорошо знает русский язык но не знает json и плохо умеет генерировать структурированные данные (он на них не учился) поэтому следующий этап - делать промежуточное преобразование в псевдокод и обратно, там мы избежим поломки структуры


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls -la /content/drive/MyDrive/VKR/


total 16
drwx------ 2 root root 4096 Apr 27 05:50 dataset
drwx------ 2 root root 4096 Apr 27 05:53 library
drwx------ 2 root root 4096 Apr 29 09:32 metrics_T5ru
drwx------ 2 root root 4096 Apr 29 09:32 T5ru_lora_outputs


In [3]:
!pip install -U spacy > /dev/null 2>&1
!python -m spacy download ru_core_news_sm > /dev/null 2>&1
!pip install wandb > /dev/null 2>&1
!pip install datasets > /dev/null 2>&1

In [4]:
import transformers
import datasets
import huggingface_hub
import torch
import wandb

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(torch.__version__)

4.51.3
3.5.1
0.30.2
2.6.0+cu124


Будем напрямую генерировать json по сцене, для этого дообучим T5(Text-To-Text Transfer Transformer) + LoRA

In [5]:
import json
import torch
import os
import sys
import warnings
import random

import numpy as np

from pathlib import Path
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import TrainerCallback

from peft import LoraConfig, get_peft_model, TaskType, PeftConfig,PeftModel
from tqdm import tqdm

import matplotlib.pyplot as plt

# отключаем их все чтобы картинку не портили
warnings.filterwarnings("ignore", category=FutureWarning)

DATA_DIR = Path("/content/drive/MyDrive/VKR/dataset/dataset_tmp").expanduser()
#DATA_DIR = Path("/content/drive/MyDrive/VKR/dataset/dataset_small").expanduser()
MODEL_NAME = "sberbank-ai/ruT5-base"

lib_path = os.path.abspath(os.path.join(os.getcwd(), '/content/drive/MyDrive/VKR/'))
sys.path.append(lib_path)

from library.metrics_pseudo import evaluate_all_metrics
from library.safe_compute_metrics import safe_compute_metrics

# перевод в псевдотекст и обратно
from library.utils import json_to_pseudo_text, pseudo_text_to_json

### Промпт с "few shorts" примерами

In [6]:
# промпт очень большой, поэтому нужно чтобы все влезало
PROMPT = """
Ты должен проанализировать описание сцены и вернуть ответ в специальном псевдоформате.

Твоя задача:
- Найди все объекты, упомянутые в описании, и их признаки.
- Верни результат строго в псевдоформате — одной строкой.

Формат:
объект1 (признак1 признак2) объект2 () объект3 (признак)

Требования:
- Каждый объект указывается один раз.
- Признаки пишутся через пробел внутри круглых скобок.
- Если признаки отсутствуют, используй пустые скобки ().
- Не добавляй объектов или признаков, которых нет в описании.
- В ответе не должно быть никаких пояснений, комментариев или заголовков — только одна строка с результатом.

Примеры:

Описание: Маленький красный стол стоит у окна.
Ответ:
стол (маленький красный) окно ()

Описание: {description}

Ответ:
"""

print(len(PROMPT))

750


        # Логируем метрики
        # 'f1_objects': только по объектам,
        # 'f1_attributes_macro': Это среднее значение F1 по атрибутам (признакам),
        #                        рассчитанное отдельно для каждого объекта,
        #                        а потом усреднённое по всем объектам.
        #'f1_attributes_weighted': То же, но взвешенное по числу признаков в эталоне
        #'f1_global_obj_attr_pairs': F1 по всем (объект, признак) парам как единому множеству
        #'f1_combined_simple': простое среднее между двух ключевых компонент качества (F1 объекты и F1 признаки)
        #'f1_combined_weighted': взвешенное среднее двух F1-метрик с учетом числа объектов и числа признаков

In [7]:
def postprocess_text(preds, labels):
    preds_json = []
    labels_json = []

    for pred_str, label_str in zip(preds, labels):
        try:
            pred_json = pseudo_text_to_json(pred_str.strip())
        except Exception as e:
            #print("Ошибка парсинга предсказания:", pred_str, "|", e)
            pred_json = []

        try:
            label_json = pseudo_text_to_json(label_str.strip())
        except Exception as e:
            #print("Ошибка парсинга ground truth:", label_str, "|", e)
            label_json = []

        preds_json.append(pred_json)
        labels_json.append(label_json)

    # посмотрим что выдает
    print(preds_json[0], labels_json[0])

    return preds_json, labels_json

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Если predictions — logits, нужно брать argmax
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    # logits -> ids
    predictions = np.argmax(predictions, axis=-1)

    # Декодируем токены
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Превращаем строки в JSON-списки через внешний postprocess_text
    preds_json, labels_json = postprocess_text(decoded_preds, decoded_labels)

    # Метрики
    f1_objects_list = []
    f1_attributes_macro_list = []
    f1_attributes_weighted_list = []
    f1_combined_simple_list = []
    f1_combined_weighted_list = []
    f1_global_obj_attr_pairs_list = []

    valid = 0

    for pred, label in zip(preds_json, labels_json):

        #print("pred:", pred, type(pred))
        #print("label:", label, type(label))
        #print("-------------------------------")

        if not isinstance(pred, list) or not isinstance(label, list):
            f1_objects_list.append(0.0)
            f1_attributes_macro_list.append(0.0)
            f1_attributes_weighted_list.append(0.0)
            f1_combined_simple_list.append(0.0)
            f1_combined_weighted_list.append(0.0)
            f1_global_obj_attr_pairs_list.append(0.0)
            #print("bad evaluation")
            continue

        try:
            scores = evaluate_all_metrics(label, pred)
            #print("good:",scores)
            f1_objects_list.append(scores["f1_objects"])
            f1_attributes_macro_list.append(scores["f1_attributes_macro"])
            f1_attributes_weighted_list.append(scores["f1_attributes_weighted"])
            f1_combined_simple_list.append(scores["f1_combined_simple"])
            f1_combined_weighted_list.append(scores["f1_combined_weighted"])
            f1_global_obj_attr_pairs_list.append(scores["f1_global_obj_attr_pairs"])
            valid += 1

        except Exception as e:
            f1_objects_list.append(0.0)
            f1_attributes_macro_list.append(0.0)
            f1_attributes_weighted_list.append(0.0)
            f1_combined_simple_list.append(0.0)
            f1_combined_weighted_list.append(0.0)
            f1_global_obj_attr_pairs_list.append(0.0)

    total = len(decoded_preds)

    aggregated_scores = {
        "f1_objects": round(sum(f1_objects_list) / total, 4),
        "f1_attributes_macro": float(round(sum(f1_attributes_macro_list) / total, 4)),
        "f1_attributes_weighted": float(round(sum(f1_attributes_weighted_list) / total, 4)),
        "f1_combined_simple": float(round(sum(f1_combined_simple_list) / total, 4)),
        "f1_combined_weighted": float(round(sum(f1_combined_weighted_list) / total, 4)),
        "f1_global_obj_attr_pairs": float(round(sum(f1_global_obj_attr_pairs_list) / total, 4)),
        "valid_json_rate": float(round(valid / total, 4)),
        "total_samples": float(total),
        "valid_samples": float(valid),
    }
    print("aggregated_scores:", aggregated_scores)
    return aggregated_scores

### Параметры модели и обучения

инъекции будем делать во все слои связанные с вниманием - это должно сделать модель гибче

In [8]:
lora_rank = 8
lora_alpha = 16
lora_target_modules=["q", "v"]   # в какие слои делаем инъекции
lora_dropout=0.1

per_device_train_batch_size = 8
num_train_epochs = 50

INPUT_SEQ_LENGTH = 1100
OUTPUT_SEQ_LENGTH = 512

# параметры генерации
NUM_BEAMS = 6

# лучше beam подлиннее чем температура, тк у нас структурированный текст
#TEMPERATURE = 0.7

In [9]:
run = wandb.init(
    entity="shiltsov-da",
    # Set the wandb project where this run will be logged.
    project="vkr-hse-object-detection",
    # Track hyperparameters and run metadata.
    group="T5LoRAtext2textPsC",
    tags=["text2text", "lora", MODEL_NAME],
    config={
        "architecture": "FlanT5-LoRA-text2text-PsC",
        "notebook":"T5ru-LoRA-text2text-parser-v3-Colab.ipynb",
        "base_model": MODEL_NAME,
        "lora_rank": lora_rank,
        "lora_alpha": lora_alpha,
        "lora_target_modules": lora_target_modules,
        "per_device_train_batch_size": per_device_train_batch_size,
        "num_train_epochs": num_train_epochs
    },
)



<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shiltsov (shiltsov-da) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:
# Грузим батчи
def make_target(scene_objects):
    objects_dict = {}
    for obj in scene_objects:
        for name, attrs in obj.items():
            objects_dict[name] = attrs
    ps_text = json_to_pseudo_text([objects_dict])
    return ps_text

data = []
for path in sorted(DATA_DIR.glob("*.jsonl")):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            description = item["description"]
            target = make_target(item["scene"]["objects"])
            data.append({
                "input": PROMPT.format(description=description),
                "target": target,
            })


# Делаем датасет
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_ds, val_ds = dataset["train"], dataset["test"]

print(train_ds[0])

{'input': '\nТы должен проанализировать описание сцены и вернуть ответ в специальном псевдоформате.\n\nТвоя задача:\n- Найди все объекты, упомянутые в описании, и их признаки.\n- Верни результат строго в псевдоформате — одной строкой.\n\nФормат:\nобъект1 (признак1 признак2) объект2 () объект3 (признак)\n\nТребования:\n- Каждый объект указывается один раз.\n- Признаки пишутся через пробел внутри круглых скобок.\n- Если признаки отсутствуют, используй пустые скобки ().\n- Не добавляй объектов или признаков, которых нет в описании.\n- В ответе не должно быть никаких пояснений, комментариев или заголовков — только одна строка с результатом.\n\nПримеры:\n\nОписание: Маленький красный стол стоит у окна.\nОтвет:\nстол (маленький красный) окно ()\n\nОписание: Чемодан стоит рядом с цветным телевизором, возле которого лежит ключ.\n\nОтвет:\n', 'target': 'ключ () телевизор (цветной) чемодан ()'}


In [11]:
# Токенайзер
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def preprocess(example):
    inputs = tokenizer(example["input"], padding="max_length", truncation=True, max_length=INPUT_SEQ_LENGTH)
    targets = tokenizer(example["target"], padding="max_length", truncation=True, max_length=OUTPUT_SEQ_LENGTH)
    inputs["labels"] = targets["input_ids"]

    # сохраняем оригинал обратно в пример
    # inputs["target_raw"] = example["target_raw"]
    return inputs

train_ds = train_ds.map(preprocess, batched=False)
val_ds = val_ds.map(preprocess, batched=False)

# Грузим модель + LoRA
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

lora_config = LoraConfig(
    r=lora_rank, # ранг низкоранговой матрицы
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    # target_modules=["q", "k", "v", "o"]
    lora_dropout=lora_dropout,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)

# Обучение
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/VKR/T5ru_PsC_lora_outputs",
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=4,
    num_train_epochs=num_train_epochs,
    logging_dir="/content/drive/MyDrive/VKR/logs_T5ru_PsC",
    logging_steps=50,
    eval_strategy="epoch",
    eval_accumulation_steps=10, # для маленькой памяти GPU - ск бачей одновременно грузить
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="wandb",
    fp16=True
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# грузим уже обученную
#trainer.train(resume_from_checkpoint=True)

trainer.train()
model.save_pretrained("/content/drive/MyDrive/VKR/T5ru_PsC_lora_outputs")
run.finish()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Epoch,Training Loss,Validation Loss,F1 Objects,F1 Attributes Macro,F1 Attributes Weighted,F1 Combined Simple,F1 Combined Weighted,F1 Global Obj Attr Pairs,Valid Json Rate,Total Samples,Valid Samples
1,0.164300,0.089051,0.245000,0.005200,0.015800,0.125100,0.231200,0.007800,1.000000,253.000000,253.000000
2,0.090000,0.049482,0.513800,0.071300,0.166200,0.292600,0.437600,0.125100,1.000000,253.000000,253.000000
3,0.072300,0.040061,0.541600,0.140100,0.291800,0.340900,0.439400,0.217500,1.000000,253.000000,253.000000
4,0.062000,0.033005,0.569100,0.181100,0.370700,0.375100,0.484100,0.279400,1.000000,253.000000,253.000000
5,0.056200,0.028521,0.589800,0.241400,0.438900,0.415600,0.513300,0.350700,1.000000,253.000000,253.000000
6,0.047400,0.025599,0.641500,0.278500,0.477300,0.460000,0.556300,0.398300,1.000000,253.000000,253.000000
7,0.042000,0.024079,0.663700,0.302200,0.511500,0.483000,0.581000,0.425500,1.000000,253.000000,253.000000
8,0.039500,0.019308,0.727200,0.359400,0.544600,0.543300,0.630000,0.484100,1.000000,253.000000,253.000000
9,0.038700,0.019200,0.738800,0.371400,0.555200,0.555100,0.650500,0.496300,1.000000,253.000000,253.000000
10,0.034200,0.016282,0.771200,0.400900,0.573300,0.586000,0.669100,0.529000,1.000000,253.000000,253.000000


[{'сиденьеь': []}, {'стул': []}, {'(': []}] [{'сиденье': ['мягкое']}, {'поручень': ['металлический', 'длинный', 'гладкий']}, {'окно': ['прозрачное', 'большое']}, {'дверь': []}]
aggregated_scores: {'f1_objects': 0.245, 'f1_attributes_macro': 0.0052, 'f1_attributes_weighted': 0.0158, 'f1_combined_simple': 0.1251, 'f1_combined_weighted': 0.2312, 'f1_global_obj_attr_pairs': 0.0078, 'valid_json_rate': 1.0, 'total_samples': 253.0, 'valid_samples': 253.0}
[{'сиденье': []}, {'дверьь': []}, {'дверь': []}, {'дверь': []}] [{'сиденье': ['мягкое']}, {'поручень': ['металлический', 'длинный', 'гладкий']}, {'окно': ['прозрачное', 'большое']}, {'дверь': []}]
aggregated_scores: {'f1_objects': 0.5138, 'f1_attributes_macro': 0.0713, 'f1_attributes_weighted': 0.1662, 'f1_combined_simple': 0.2926, 'f1_combined_weighted': 0.4376, 'f1_global_obj_attr_pairs': 0.1251, 'valid_json_rate': 1.0, 'total_samples': 253.0, 'valid_samples': 253.0}
[{'сиденье': ['гладгкое']}, {'дверь': ['прозрачное']}] [{'сиденье': ['мяг

Epoch,Training Loss,Validation Loss,F1 Objects,F1 Attributes Macro,F1 Attributes Weighted,F1 Combined Simple,F1 Combined Weighted,F1 Global Obj Attr Pairs,Valid Json Rate,Total Samples,Valid Samples
1,0.164300,0.089051,0.245000,0.005200,0.015800,0.125100,0.231200,0.007800,1.000000,253.000000,253.000000
2,0.090000,0.049482,0.513800,0.071300,0.166200,0.292600,0.437600,0.125100,1.000000,253.000000,253.000000
3,0.072300,0.040061,0.541600,0.140100,0.291800,0.340900,0.439400,0.217500,1.000000,253.000000,253.000000
4,0.062000,0.033005,0.569100,0.181100,0.370700,0.375100,0.484100,0.279400,1.000000,253.000000,253.000000
5,0.056200,0.028521,0.589800,0.241400,0.438900,0.415600,0.513300,0.350700,1.000000,253.000000,253.000000
6,0.047400,0.025599,0.641500,0.278500,0.477300,0.460000,0.556300,0.398300,1.000000,253.000000,253.000000
7,0.042000,0.024079,0.663700,0.302200,0.511500,0.483000,0.581000,0.425500,1.000000,253.000000,253.000000
8,0.039500,0.019308,0.727200,0.359400,0.544600,0.543300,0.630000,0.484100,1.000000,253.000000,253.000000
9,0.038700,0.019200,0.738800,0.371400,0.555200,0.555100,0.650500,0.496300,1.000000,253.000000,253.000000
10,0.034200,0.016282,0.771200,0.400900,0.573300,0.586000,0.669100,0.529000,1.000000,253.000000,253.000000


[{'сиденье': ['мягкое']}, {'поручень': ['металлический', 'длинный', 'гладкий']}, {'дверь': ['прозрачное', 'большое']}, {'дверь': []}] [{'сиденье': ['мягкое']}, {'поручень': ['металлический', 'длинный', 'гладкий']}, {'окно': ['прозрачное', 'большое']}, {'дверь': []}]
aggregated_scores: {'f1_objects': 0.8982, 'f1_attributes_macro': 0.6, 'f1_attributes_weighted': 0.7853, 'f1_combined_simple': 0.7491, 'f1_combined_weighted': 0.8373, 'f1_global_obj_attr_pairs': 0.7625, 'valid_json_rate': 1.0, 'total_samples': 253.0, 'valid_samples': 253.0}
[{'сиденье': ['мягкое']}, {'поручень': ['гладлический', 'длинный', 'гладкий']}, {'дверь': ['прозрачное', 'большое']}, {'дверь': []}] [{'сиденье': ['мягкое']}, {'поручень': ['металлический', 'длинный', 'гладкий']}, {'окно': ['прозрачное', 'большое']}, {'дверь': []}]
aggregated_scores: {'f1_objects': 0.9035, 'f1_attributes_macro': 0.6068, 'f1_attributes_weighted': 0.7935, 'f1_combined_simple': 0.7552, 'f1_combined_weighted': 0.8419, 'f1_global_obj_attr_pair

eval/f1_attributes_macro,▁▂▃▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█▇▇███████████████
eval/f1_attributes_weighted,▁▂▃▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇████████████████████
eval/f1_combined_simple,▁▃▃▄▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████████████
eval/f1_combined_weighted,▁▃▃▄▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████
eval/f1_global_obj_attr_pairs,▁▂▃▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████████████████
eval/f1_objects,▁▄▄▄▅▆▆▇▇▇▇▇▇▇▇█████████████████████████
eval/loss,█▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▃▃▄▃▆▆▄▆▆▇▇▇▆█▅▇▇▇▇▇▇██▅▇██▆▇▇█▇█▇██▅▆█
eval/samples_per_second,█▆▆▅▅▃▃▅▃▃▂▂▂▃▁▄▂▂▂▁▂▂▃▁▁▂▂▁▁▃▂▁▂▁▂▁▁▄▂▁
eval/steps_per_second,█▆▆▅▆▃▃▅▃▃▂▂▂▂▃▄▂▂▂▂▂▂▃▁▁▂▂▁▁▃▂▂▁▂▁▃▁▄▃▁
eval/total_samples,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [12]:
run.finish()

### Проверка

In [13]:
MODEL_DIR = "/content/drive/MyDrive/VKR/T5ru_PsC_lora_outputs"  # путь к fine-tuned модели
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка модели и токенизатора
print("Loading model...")
config = PeftConfig.from_pretrained(MODEL_DIR)
base_model = T5ForConditionalGeneration.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, MODEL_DIR)
model = model.to(DEVICE)
model.eval()

tokenizer = T5Tokenizer.from_pretrained(config.base_model_name_or_path)

# Генерация
def predict(description, max_length=OUTPUT_SEQ_LENGTH):
    prompt = PROMPT.format(description=description)
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=INPUT_SEQ_LENGTH
    ).to(DEVICE)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_beams=NUM_BEAMS, # попробовать меньше
            #temperature=TEMPERATURE, # параметризовать
            early_stopping=True
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    #print(output_text)
    #print(pseudo_text_to_json(output_text))
    try:
        parsed_json = pseudo_text_to_json(output_text)
    except Exception as e:
        print(f"Ошибка парсинга JSON: {e}")
        print("Сырые данные:", output_text)
        parsed_json = None

    return parsed_json


text = input("Введите описание сцены: ")
result = predict(text)
print("\nПредсказание:\n")
print(json.dumps(result, indent=2, ensure_ascii=False))


Loading model...
Введите описание сцены: Черный кот сидел рядом с пустой кастрюлей

Предсказание:

[
  {
    "кот": [
      "чёрный"
    ]
  },
  {
    "кастрюля": [
      "пустая"
    ]
  }
]


In [ ]:
pseudo_text_to_json("кот (черный) кот (черный) кот (черный) дерево (красное) дерево ()")

## Просмотр сколько параметров учили

In [14]:
def print_trainable_parameters(model):
    trainable_params = 0
    total_params = 0

    for param in model.parameters():
        total_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"Всего параметров: {total_params / 1e6:.2f}M")
    print(f"Обучаемых параметров: {trainable_params / 1e6:.2f}M")
    print(f"Доля обучаемых параметров: {100 * trainable_params / total_params:.2f}%")

# Вызов функции после создания модели

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

Всего параметров: 223.79M
Обучаемых параметров: 0.88M
Доля обучаемых параметров: 0.40%


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'sberbank-ai/ruT5-base' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
